# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [180]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [181]:
import numpy as np
import random
import math

def solve_tsp_nearest_neighbour(instance):
    first  = random.randint(0, len(instance) - 1) # Первую вершину цикла выберем произвольно
    answer = [0]*len(instance)
    is_visited = np.zeros(len(instance)) # Посещены ли вершины?
    is_visited[first] = 1
    answer[0] = first
    for i in range(0, len(instance) - 1): 
        distance = math.inf
        nearest = 0
        for j in range(0, len(instance)): # Выбираем ближайшего среди непосещенных
            cur_distance = euclidean_distance(instance[answer[i]], instance[j])
            if(is_visited[j] == 0 and cur_distance < distance):
                distance = cur_distance 
                nearest = j
        answer[i + 1] = nearest
        is_visited[nearest] = 1 
    return answer

In [185]:
import numpy as np
import random
import math


def solve_tsp_nearest_insertion(instance):
    
    def find_first_vertices(): # Находим пару вершин, соединённую самым коротким ребром
        cur_distance = math.inf
        for i in range(0, len(instance)):
            for j in range(0, len(instance)):
                d = euclidean_distance(instance[i], instance[j])
                if i != j and cur_distance > d:
                    cur_distance = d
                    answer[0] = i
                    answer[1] = j
        is_visited[answer[0]] = 1
        is_visited[answer[1]] = 1
    

    def find_nearest(): # Находим ближайшую к выбранным вершину
        cur_nearest = -1
        cur_distance = math.inf
            
        vertex = answer[len(answer) - 1]
        for j in range(0, len(instance)): # Обновляем расстояния до вершин
            if(is_visited[j] == 0):
                d = euclidean_distance(instance[j], instance[vertex])
                if is_visited[j] == 0 and cur_min_distance[j] > d:
                    cur_min_distance[j] = d
                    
        for j in range(0, len(instance)): # Ищем минимум
            if is_visited[j] == 0 and cur_min_distance[j] < cur_distance:
                cur_distance = cur_min_distance[j]
                cur_nearest = j
                
        is_visited[cur_nearest] = 1
        return cur_nearest
    

    def find_best_place(vertex): # Ищем лучшее место для вставки новой вершины
        cur_nearest = -1
        cur_distance = math.inf
        for k in range(0, len(answer) - 1):
            d = euclidean_distance(instance[answer[k]], instance[vertex]) + \
                                euclidean_distance(instance[answer[k + 1]], instance[vertex])
            if cur_distance > d:
                cur_distance = d
                cur_nearest = k
        return cur_nearest
    

    answer = [0]*2
    cur_min_distance = [math.inf]*len(instance) # Текущие минимальные расстояния до всех вершин графа от выбранных
    is_visited = np.zeros(len(instance))
    find_first_vertices()
    for i in range(0, len(instance) - 2): # Выбираем следующую вершину
        new_vertex = find_nearest()
        answer.insert(find_best_place(new_vertex) + 1, new_vertex)
    return answer

In [186]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [187]:
run_all()

Solving instance d198.tsp… done in 0.2 seconds with tour length 18506 using NN and in 0.45 seconds with tour length 19944 using NI
Solving instance d493.tsp… done in 1.2 seconds with tour length 43699 using NN and in 2.5 seconds with tour length 48762 using NI
Solving instance d657.tsp… done in 1.4 seconds with tour length 65173 using NN and in 2.3 seconds with tour length 73468 using NI
Solving instance d2103.tsp… done in 5.2 seconds with tour length 94185 using NN and in 1.5e+01 seconds with tour length 137254 using NI
Solving instance pr107.tsp… done in 0.016 seconds with tour length 51452 using NN and in 0.031 seconds with tour length 54492 using NI
Solving instance pr152.tsp… done in 0.031 seconds with tour length 88823 using NN and in 0.063 seconds with tour length 106938 using NI
Solving instance pr439.tsp… done in 0.23 seconds with tour length 131294 using NN and in 1.3 seconds with tour length 173907 using NI


## Выводы
(Опишите в 1-2 предложениях свои наблюдения по результатам запусков.)

**Выводы:** на данных примерах алгоритм NN показал результат лучше, чем NI. Результаты NI меньше не более чем на 30%, то есть разница не слишком большая. То есть алгоритмы работают почти одинаково.